In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct, sum, avg, when

# Initialize Spark Session
spark = SparkSession.builder.appName("SparkSQL_Assignment").getOrCreate()

train_df = spark.read.csv("hdfs://m01.itversity.com:9000/user/ana017398/train.csv", header=True, inferSchema=True)
fulfillment_df = spark.read.csv("hdfs://m01.itversity.com:9000/user/ana017398/fulfilment_center_info.csv", header=True, inferSchema=True)
meal_df = spark.read.csv("hdfs://m01.itversity.com:9000/user/ana017398/meal_info.csv", header=True, inferSchema=True)


# Register DataFrames as SQL tables
train_df.createOrReplaceTempView("train")
fulfillment_df.createOrReplaceTempView("fulfillment")
meal_df.createOrReplaceTempView("meal")

In [12]:
#Q1

In [4]:
spark.sql("""
SELECT COUNT(DISTINCT category) AS distinct_categories, 
       COUNT(DISTINCT cuisine) AS distinct_cuisines 
FROM meal
""").show()


+-------------------+-----------------+
|distinct_categories|distinct_cuisines|
+-------------------+-----------------+
|                 14|                4|
+-------------------+-----------------+



In [13]:
#Q2

In [5]:
spark.sql("""
SELECT center_id, SUM(num_orders) AS total_orders
FROM train
GROUP BY center_id
ORDER BY total_orders DESC
LIMIT 1
""").show()

+---------+------------+
|center_id|total_orders|
+---------+------------+
|       13|     1742220|
+---------+------------+



In [14]:
#Q3

In [6]:
spark.sql("""
WITH TopCenter AS (
    SELECT center_id
    FROM train
    GROUP BY center_id
    ORDER BY SUM(num_orders) DESC
    LIMIT 1
)
SELECT m.cuisine, SUM(t.num_orders) AS total_orders
FROM train t
JOIN meal m ON t.meal_id = m.meal_id
JOIN TopCenter tc ON t.center_id = tc.center_id
GROUP BY m.cuisine
ORDER BY total_orders DESC
LIMIT 1
""").show()

+-------+------------+
|cuisine|total_orders|
+-------+------------+
|   Thai|      654724|
+-------+------------+



In [15]:
#Q4

In [7]:
spark.sql("""
SELECT center_type, AVG(op_area) AS avg_op_area
FROM fulfillment
GROUP BY center_type
""").show()

+-----------+------------------+
|center_type|       avg_op_area|
+-----------+------------------+
|     TYPE_C|3.1578947368421044|
|     TYPE_B|4.7733333333333325|
|     TYPE_A| 4.076744186046512|
+-----------+------------------+



In [16]:
#Q5

In [8]:
spark.sql("""
SELECT f.center_type, SUM(t.checkout_price * t.num_orders) AS total_revenue
FROM train t
JOIN fulfillment f ON t.center_id = f.center_id
GROUP BY f.center_type
ORDER BY total_revenue DESC
LIMIT 1
""").show()

+-----------+-------------------+
|center_type|      total_revenue|
+-----------+-------------------+
|     TYPE_A|7.276203201869873E9|
+-----------+-------------------+



In [17]:
#Q6

In [9]:
spark.sql("""
SELECT m.cuisine, SUM(t.num_orders) AS total_orders
FROM train t
JOIN meal m ON t.meal_id = m.meal_id
GROUP BY m.cuisine
ORDER BY total_orders DESC
LIMIT 1
""").show()

+-------+------------+
|cuisine|total_orders|
+-------+------------+
|Italian|    17166334|
+-------+------------+



In [18]:
#Q7

In [10]:
spark.sql("""
SELECT t.week, m.cuisine, SUM(t.num_orders) AS total_orders
FROM train t
JOIN meal m ON t.meal_id = m.meal_id
GROUP BY t.week, m.cuisine
ORDER BY t.week, total_orders DESC
""").show()

+----+-----------+------------+
|week|    cuisine|total_orders|
+----+-----------+------------+
|   1|       Thai|      242088|
|   1|    Italian|      228836|
|   1|     Indian|      175317|
|   1|Continental|      146020|
|   2|       Thai|      273778|
|   2|    Italian|      202627|
|   2|     Indian|      177109|
|   2|Continental|      133570|
|   3|       Thai|      249838|
|   3|    Italian|      197299|
|   3|     Indian|      150148|
|   3|Continental|       97977|
|   4|       Thai|      277206|
|   4|    Italian|      192265|
|   4|     Indian|      155239|
|   4|Continental|      118819|
|   5|     Indian|      683532|
|   5|       Thai|      229905|
|   5|    Italian|      169161|
|   5|Continental|      116077|
+----+-----------+------------+
only showing top 20 rows



In [19]:
#Q8

In [11]:
spark.sql("""
SELECT center_id, COUNT(*) AS discount_count
FROM train
WHERE checkout_price < base_price
GROUP BY center_id
ORDER BY discount_count DESC
LIMIT 1
""").show()

+---------+--------------+
|center_id|discount_count|
+---------+--------------+
|       13|          1509|
+---------+--------------+

